In [5]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import sys

In [8]:
PROJECT_ROOT = Path().resolve().parents[0]
sys.path.append(str(PROJECT_ROOT))
print(PROJECT_ROOT)

D:\Code_ML\energy_consumption_forecasting


In [ ]:
from src.data.load_data import load_raw_data

In [10]:
folder_path = "D:/Code_ML/energy_consumption_forecasting/src/data/raw"
    

In [11]:
df = load_raw_data(Path(folder_path))
df.head()

,Datetime,AEP_MW,utility,DAYTON_MW,DOM_MW,DUQ_MW
2183,2004-10-01 01:00:00,12379.0,AEP,NaN,NaN,NaN
123456,2004-10-01 01:00:00,NaN,DAYTON,1621.0,NaN,NaN
2184,2004-10-01 02:00:00,11935.0,AEP,NaN,NaN,NaN
123457,2004-10-01 02:00:00,NaN,DAYTON,1536.0,NaN,NaN
123458,2004-10-01 03:00:00,NaN,DAYTON,1500.0,NaN,NaN
